In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import gmaps.datasets
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (Inches),Snow (Inches)
0,0,Tiruchchendur,IN,8.48,78.12,81.25,71,100,12.66,moderate rain,1.2,0
1,1,Yellowknife,CA,62.46,-114.35,64.40,45,40,5.82,scattered clouds,0.0,0
2,2,College,US,64.86,-147.80,57.99,76,90,2.42,overcast clouds,0.0,0
3,3,Bulawayo,ZW,-20.15,28.58,55.13,62,48,14.74,scattered clouds,0.0,0
4,4,Ushuaia,AR,-54.80,-68.30,41.00,75,40,8.05,scattered clouds,0.0,0


In [ ]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [ ]:
# Ask the customer to add raining and snowing preferences.
rain_input = str(input("Do you want it to be raining? (yes/no) "))
snow_input = str(input("Do you want it to be snowing? (yes/no) "))

In [ ]:
# Filter the dataset to find the cities that fit the criteria.
if rain_input == "yes" and snow_input =="yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (Inches)"] > 0) & \
                                           (city_data_df["Snow (Inches)"] > 0)]   
elif rain_input == "yes" and snow_input =="no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (Inches)"] > 0) & \
                                           (city_data_df["Snow (Inches)"] == 0)] 
elif rain_input == "no" and snow_input =="no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (Inches)"] == 0) & \
                                           (city_data_df["Snow (Inches)"] == 0)] 
elif rain_input == "no" and snow_input =="yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (Inches)"] == 0) & \
                                           (city_data_df["Snow (Inches)"] > 0)]
else:
    print("Please enter either yes or no response")


In [ ]:
preferred_cities_df.head(10)

In [ ]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and current weather description.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}

In [ ]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [ ]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [ ]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [ ]:
hotel_df.head(10)

In [ ]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F </dd>
</dl>
"""

In [ ]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


In [ ]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the WeatherPy_vacation into a CSV.
hotel_df.to_csv(output_data_file)